In [ ]:
from keras.models import Model, load_model
from keras.layers import Dense
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.utils import np_utils

# Load the dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# One hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Reshape to be [samples][width][height][channels]
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32')

# Build the model
model = Sequential()

# 1st convolution layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten layer
model.add(Flatten())

# Fully connected layer
model.add(Dense(128, activation='relu'))

# Additional layer for key generation
model.add(Dense(128, activation='tanh'))

# Output layer
model.add(Dense(10, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

# Save the model
model.save("mnist_cnn_model.h5")




Epoch 1/10
300/300 [==============================] - 33s 108ms/step - loss: 0.5806 - accuracy: 0.7866 - val_loss: 0.4707 - val_accuracy: 0.8142
Epoch 2/10
300/300 [==============================] - 32s 108ms/step - loss: 0.3606 - accuracy: 0.8676 - val_loss: 0.3498 - val_accuracy: 0.8749
Epoch 3/10
300/300 [==============================] - 31s 104ms/step - loss: 0.3087 - accuracy: 0.8864 - val_loss: 0.3080 - val_accuracy: 0.8872
Epoch 4/10
300/300 [==============================] - 32s 106ms/step - loss: 0.2748 - accuracy: 0.8978 - val_loss: 0.2897 - val_accuracy: 0.8911
Epoch 5/10
300/300 [==============================] - 32s 105ms/step - loss: 0.2497 - accuracy: 0.9078 - val_loss: 0.2766 - val_accuracy: 0.8992
Epoch 6/10
300/300 [==============================] - 31s 104ms/step - loss: 0.2323 - accuracy: 0.9138 - val_loss: 0.2596 - val_accuracy: 0.9051
Epoch 7/10
300/300 [==============================] - 32s 108ms/step - loss: 0.2166 - accuracy: 0.9191 - val_loss: 0.2657 - val_ac

In [ ]:
# Load the model
model = load_model("mnist_cnn_model.h5")

# Create a new model that shares all layers up to and including the new dense layer with the original model
key_gen_model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

def generate_key(image):
    # Make sure the image is normalized and in the right shape
    image = image / 255
    image = image.reshape((1, 28, 28, 1))

    # Generate the key
    key = key_gen_model.predict(image)

    # Scale the key's elements to be between 0 and 255
    key = (key - key.min()) / (key.max() - key.min()) * 255

    # Round the key's elements to the nearest integers
    key = np.round(key)

    # Reshape the key to a 1D array
    key = key.flatten()

    # Truncate the key to be the length of row*col of the image
    key = key[:28*28]

    # # Reshape the key back to a 2D array
    # key = key.reshape((28, 28))

    return key

# Test the function with a sample image
sample_image = X_train[0]
key = generate_key(sample_image)

1/1 [==============================] - 0s 66ms/step


In [ ]:
key

array([163., 217., 144.,   0.,  88., 112.,  24., 166., 230., 163., 159.,
       123., 172.,  66.,  40.,  76., 161.,  44.,  72.,   1., 141., 146.,
       152., 215., 237., 185., 149., 112., 171.,  92., 112., 103., 114.,
       105., 162., 154.,  66.,  63.,  90., 186., 190., 227., 194., 125.,
       114., 160., 144.,  70., 125., 189.,  62.,  89., 115., 188., 177.,
       165.,  39., 192., 171., 157., 149., 137.,  57., 139.,  72., 204.,
        31., 186.,  82., 193., 175.,  32.,  54., 115., 143., 111., 150.,
       233., 102.,  97.,  81.,  79., 226., 168.,  70., 138.,  39., 137.,
        48., 207., 144., 162.,  89., 107.,  86., 158., 222., 138.,  95.,
       178., 145., 148., 179.,  92.,  54.,  37., 224.,  96., 107., 250.,
       223., 191., 240., 139.,  82.,  36., 204., 215.,  82.,  55., 134.,
       165., 255., 134.,  52.,  67.,  97., 140.], dtype=float32)